In [10]:
from openmm import unit
from openmmtools import testsystems, states, mcmc
import tempfile
testsystem = testsystems.AlanineDipeptideImplicit()
import os
from openmmtools.multistate import ParallelTemperingSampler
from openmmtools.multistate import MultiStateReporter


In [2]:
n_replicas = 3  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 600.0 * unit.kelvin  # Maximum temperature.
reference_state = states.ThermodynamicState(system=testsystem.system, temperature=T_min)


In [5]:
move = mcmc.GHMCMove(timestep=2.0*unit.femtoseconds, n_steps=50)
simulation = ParallelTemperingSampler(mcmc_moves=move, number_of_iterations=2)


In [11]:
storage_path = 'parallel_tempering.nc'
reporter = MultiStateReporter(storage_path, checkpoint_interval=10)
simulation.create(reference_state,
                  states.SamplerState(testsystem.positions),
                  reporter, min_temperature=T_min,
                  max_temperature=T_max, n_temperatures=n_replicas)

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

In [ ]:
simulation

In [27]:
simulation.run(n_iterations=1000)

In [14]:
from netCDF4 import Dataset


In [20]:
dataset = Dataset(storage_path, mode="r")

In [21]:
dataset

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    UUID: de45d624-c241-4f73-afb8-fe6f12c90d69
    program: openmmtools 0.23.1
    programVersion: 0.23.1
    Conventions: ReplicaExchange
    ConventionVersion: 0.2
    DataUsedFor: analysis
    CheckpointInterval: 10
    title: Parallel tempering simulation created using ParallelTempering class of yank.multistate on Fri Nov 15 16:38:05 2024
    dimensions(sizes): scalar(1), iteration(3), spatial(3), analysis_particles(0), fixedL5272(5272), fixedL489(489), fixedL3(3), replica(3), state(3)
    variables(dimensions): int64 last_iteration(scalar), int64 analysis_particle_indices(analysis_particles), <class 'str'> options(scalar), |S1 metadata(fixedL3), int32 states(iteration, replica), float64 energies(iteration, replica, state), int8 neighborhoods(iteration, replica, state), int32 accepted(iteration, state, state), int32 proposed(iteration, state, state), <class 'str'> timestamp(iteration)
    groups: 

In [22]:
dataset.variables.keys()

dict_keys(['last_iteration', 'analysis_particle_indices', 'options', 'metadata', 'states', 'energies', 'neighborhoods', 'accepted', 'proposed', 'timestamp'])

In [23]:
dataset['metadata']

<class 'netCDF4._netCDF4.Variable'>
|S1 metadata(fixedL3)
unlimited dimensions: 
current shape = (3,)
filling on, default _FillValue of   used

In [25]:
dataset['last_iterationl']

<class 'netCDF4._netCDF4.Variable'>
int64 last_iteration(scalar)
unlimited dimensions: 
current shape = (1,)
filling on, default _FillValue of -9223372036854775806 used